In [1]:

import os
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import numpy as np


from config import Config
from utils import Logger, AverageMeter, evaluate, data_augmentadion_localizer, diceCoeff
from dataset.dataset import BUSIDataset_image_mask_name
import segmentation_models_pytorch as smp
from networks.networks import Localizer

import matplotlib.pyplot as plt
from PIL import Image

/home/zentek/.conda/envs/cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config()
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu_id


train_set = BUSIDataset_image_mask_name(
        config.image_path, config.mask_path, config.train_mapping_path,
        transform=data_augmentadion_localizer(train=False))

test_set = BUSIDataset_image_mask_name(
    config.image_path, config.mask_path, config.test_mapping_path,
    transform=data_augmentadion_localizer(train=False))


train_loader = DataLoader(train_set,
                        batch_size=1,
                        shuffle=False,
                        pin_memory=True,
                        num_workers=config.num_workers)
test_loader = DataLoader(test_set,
                        batch_size=1,
                        shuffle=False,
                        pin_memory=True,
                        num_workers=config.num_workers)

localizer = Localizer().cuda()
localizer.load_state_dict(torch.load(config.localizer_state_path + "localizer_3.pth"))

<All keys matched successfully>

In [3]:
for step, (image, mask, label, name) in enumerate(train_loader):

    image = image.cuda()
    mask = mask.cuda()
    pre_mask = localizer(image)
    # pre_mask = torch.sigmoid(pre_mask)
    pre_mask = pre_mask.cpu().detach().numpy()[0].transpose(1, 2, 0)
    pre_mask = (pre_mask - pre_mask.min()) / (pre_mask.max() - pre_mask.min())
    mask = mask.cpu().detach().numpy()[0].transpose(1, 2, 0)
    image = image.cpu().detach().numpy()[0].transpose(1, 2, 0) * 255
    image = image.astype(np.uint8)
    name = name[0]


    
    idx = np.where(pre_mask>0.5)
    idx = np.array(idx).transpose(1, 0)
    ymin = np.min(idx[:, 0])
    ymax = np.max(idx[:, 0])
    xmin = np.min(idx[:, 1])
    xmax = np.max(idx[:, 1])
    ylength = ymax - ymin
    yadapt = int(ylength*0.2)
    ymin = max(0, ymin - yadapt)
    ymax = min(image.shape[0], ymax + yadapt)
    ymid = (ymin + ymax) // 2

    xlength = xmax - xmin
    xadapt = int(xlength*0.2)
    xmin = max(0, xmin - xadapt)
    xmax = min(image.shape[1], xmax + xadapt)
    xmid = (xmin + xmax) // 2

    ylength = ymax - ymin
    xlength = xmax - xmin
    length = max(ylength, xlength)
    ymin = ymid - length // 2
    ymax = ymid + length // 2
    xmin = xmid - length // 2
    xmax = xmid + length // 2

    if ymin < 0:
        ymax = ymax - ymin
        ymin = 0
    if xmin < 0:
        xmax = xmax - xmin
        xmin = 0
    if ymax > image.shape[0]:
        ymin = ymin - (ymax - image.shape[0])
        ymax = image.shape[0]
    if xmax > image.shape[1]:
        xmin = xmin - (xmax - image.shape[1])
        xmax = image.shape[1]


    print(name)
    print(ymin, ymax, xmin, xmax)
    print(image.shape)

    image_cut = image[ymin:ymax, xmin:xmax, :]
    mask_cut = mask[ymin:ymax, xmin:xmax, :]
    pre_mask_cut = pre_mask[ymin:ymax, xmin:xmax, :]
    
    Image.fromarray(image_cut).save("./cut_results/cut_image/" + name)
    mask_cut *= 255
    mask_cut = mask_cut.astype(np.uint8)
    Image.fromarray(mask_cut[:, :, 0], "L").save("./cut_results/cut_mask/" + name)
    pre_mask_cut *= 255
    pre_mask_cut = pre_mask_cut.astype(np.uint8)
    Image.fromarray(pre_mask_cut[:, :, 0], "L").save("./cut_results/cut_pre_mask/" + name)


000067.png
0 144 36 180
(224, 224, 3)
000039.png
13 141 44 172
(224, 224, 3)
000005.png
33 69 87 123
(224, 224, 3)
000127.png
7 135 74 202
(224, 224, 3)
000103.png
55 115 76 136
(224, 224, 3)
000091.png
33 107 62 136
(224, 224, 3)
000042.png
0 202 0 202
(224, 224, 3)
000069.png
26 142 38 154
(224, 224, 3)
000101.png
104 140 52 88
(224, 224, 3)
000083.png
0 112 54 166
(224, 224, 3)
000131.png
71 143 131 203
(224, 224, 3)
000026.png
0 120 50 170
(224, 224, 3)
000130.png
17 45 94 122
(224, 224, 3)
000079.png
0 146 59 205
(224, 224, 3)
000051.png
0 140 12 152
(224, 224, 3)
000037.png
54 90 79 115
(224, 224, 3)
000078.png
0 108 78 186
(224, 224, 3)
000056.png
0 172 52 224
(224, 224, 3)
000009.png
59 103 37 81
(224, 224, 3)
000017.png
8 172 60 224
(224, 224, 3)
000112.png
0 204 4 208
(224, 224, 3)
000108.png
52 132 75 155
(224, 224, 3)
000082.png
42 130 43 131
(224, 224, 3)
000070.png
31 101 103 173
(224, 224, 3)
000126.png
0 136 56 192
(224, 224, 3)
000128.png
0 140 33 173
(224, 224, 3)
000

In [4]:
for step, (image, mask, label, name) in enumerate(test_loader):

    image = image.cuda()
    mask = mask.cuda()
    pre_mask = localizer(image)
    # pre_mask = torch.sigmoid(pre_mask)
    pre_mask = pre_mask.cpu().detach().numpy()[0].transpose(1, 2, 0)
    pre_mask = (pre_mask - pre_mask.min()) / (pre_mask.max() - pre_mask.min())
    mask = mask.cpu().detach().numpy()[0].transpose(1, 2, 0)
    image = image.cpu().detach().numpy()[0].transpose(1, 2, 0) * 255
    image = image.astype(np.uint8)
    name = name[0]


    
    idx = np.where(pre_mask>0.5)
    idx = np.array(idx).transpose(1, 0)
    ymin = np.min(idx[:, 0])
    ymax = np.max(idx[:, 0])
    xmin = np.min(idx[:, 1])
    xmax = np.max(idx[:, 1])
    ylength = ymax - ymin
    yadapt = int(ylength*0.2)
    ymin = max(0, ymin - yadapt)
    ymax = min(image.shape[0], ymax + yadapt)
    ymid = (ymin + ymax) // 2

    xlength = xmax - xmin
    xadapt = int(xlength*0.2)
    xmin = max(0, xmin - xadapt)
    xmax = min(image.shape[1], xmax + xadapt)
    xmid = (xmin + xmax) // 2

    ylength = ymax - ymin
    xlength = xmax - xmin
    length = max(ylength, xlength)
    ymin = ymid - length // 2
    ymax = ymid + length // 2
    xmin = xmid - length // 2
    xmax = xmid + length // 2

    if ymin < 0:
        ymax = ymax - ymin
        ymin = 0
    if xmin < 0:
        xmax = xmax - xmin
        xmin = 0
    if ymax > image.shape[0]:
        ymin = ymin - (ymax - image.shape[0])
        ymax = image.shape[0]
    if xmax > image.shape[1]:
        xmin = xmin - (xmax - image.shape[1])
        xmax = image.shape[1]


    print(name)
    print(ymin, ymax, xmin, xmax)
    print(image.shape)

    image_cut = image[ymin:ymax, xmin:xmax, :]
    mask_cut = mask[ymin:ymax, xmin:xmax, :]
    pre_mask_cut = pre_mask[ymin:ymax, xmin:xmax, :]
    
    Image.fromarray(image_cut).save("./cut_results/cut_image/" + name)
    mask_cut *= 255
    mask_cut = mask_cut.astype(np.uint8)
    Image.fromarray(mask_cut[:, :, 0], "L").save("./cut_results/cut_mask/" + name)
    pre_mask_cut *= 255
    pre_mask_cut = pre_mask_cut.astype(np.uint8)
    Image.fromarray(pre_mask_cut[:, :, 0], "L").save("./cut_results/cut_pre_mask/" + name)


000019.png
13 135 40 162
(224, 224, 3)
000015.png
58 100 143 185
(224, 224, 3)
000059.png
5 111 44 150
(224, 224, 3)
000086.png
42 150 49 157
(224, 224, 3)
000073.png
33 103 88 158
(224, 224, 3)
000119.png
19 119 87 187
(224, 224, 3)
000058.png
0 148 28 176
(224, 224, 3)
000129.png
0 170 33 203
(224, 224, 3)
000038.png
0 182 41 223
(224, 224, 3)
000087.png
0 224 0 224
(224, 224, 3)
000072.png
0 88 45 133
(224, 224, 3)
000104.png
17 129 68 180
(224, 224, 3)
000040.png
30 92 33 95
(224, 224, 3)
000092.png
0 146 0 146
(224, 224, 3)
000006.png
33 65 89 121
(224, 224, 3)
000035.png
54 96 84 126
(224, 224, 3)
000066.png
47 153 51 157
(224, 224, 3)
000121.png
0 136 12 148
(224, 224, 3)
000021.png
0 106 18 124
(224, 224, 3)
000116.png
0 194 18 212
(224, 224, 3)
000115.png
2 54 60 112
(224, 224, 3)
000025.png
10 132 56 178
(224, 224, 3)
000135.png
0 112 50 162
(224, 224, 3)
000057.png
27 99 81 153
(224, 224, 3)
000295.png
0 206 10 216
(224, 224, 3)
000045.png
21 161 45 185
(224, 224, 3)
000300.